<a href="https://colab.research.google.com/github/MsrOhkwr/gpu_path_tracer_with_google_colaboratory/blob/api_test/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install pyngrok
!wget -nc https://hdrihaven.com/files/hdris/museum_of_ethnography_8k.hdr

  Created wheel for pyngrok: filename=pyngrok-2.1.6-cp36-none-any.whl size=12877 sha256=2b84e07446ec1643963fbcdcbe233316310717d0e734d81111fcd547553c66f3
  Stored in directory: /root/.cache/pip/wheels/af/98/5f/8df1ba27e6159c33c4362533ce549f670e3f36a2b2ab3e816a
Successfully built pyngrok
--2020-05-06 21:34:47--  https://hdrihaven.com/files/hdris/museum_of_ethnography_8k.hdr
Resolving hdrihaven.com (hdrihaven.com)... 104.26.3.177, 104.26.2.177, 2606:4700:20::681a:3b1, ...
Connecting to hdrihaven.com (hdrihaven.com)|104.26.3.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98631972 (94M) [application/octet-stream]
Saving to: ‘museum_of_ethnography_8k.hdr’

museum_of_ethnograp 100%[===================>]  94.06M   293MB/s    in 0.3s    

2020-05-06 21:34:48 (293 MB/s) - ‘museum_of_ethnography_8k.hdr’ saved [98631972/98631972]



In [3]:
%%file api.py

from flask import Flask, Response

from engine import Renderer

app = Flask(__name__)

renderer = Renderer()
renderer.compile()
renderer.link()
renderer.bind()

@app.route("/")
def index():
    response = Response(renderer.render())
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Content-type"] = "multipart/x-mixed-replace; boundary=frame"
    return response

Writing api.py


In [4]:
%%file engine.py

import requests

from lucid.misc.gl.glcontext import create_opengl_context
import OpenGL.GL as gl
from OpenGL.GL import shaders
from string import Template
import numpy as np
import cv2

vertices = np.array(
    [
         1,  1,
         1, -1,
        -1,  1,
        -1, -1,
    ],
    dtype=np.float32
)

class Renderer:
    def __init__(self):
        self.width, self.height = (1 * x for x in (960, 540))
        create_opengl_context((self.width, self.height))


    def compile(self):
        self.sample_per_frame = 4

        vertex_shader_src = open("vertex_shader.glsl").read()
        self.vertex_shader = shaders.compileShader(vertex_shader_src, gl.GL_VERTEX_SHADER)
        fragment_shader_src = open("fragment_shader.glsl").read()
        self.fragment_shader = shaders.compileShader(fragment_shader_src, gl.GL_FRAGMENT_SHADER)
        compute_shader_src = Template(open("compute_shader.glsl").read()).substitute(sample_max = self.sample_per_frame)
        self.compute_shader = shaders.compileShader(compute_shader_src, gl.GL_COMPUTE_SHADER)


    def link(self):
        self.raster_program = shaders.compileProgram(self.vertex_shader, self.fragment_shader)
        self.raytrace_program = shaders.compileProgram(self.compute_shader)

    
    def bind(self):
        input_img_id = gl.glGenTextures(1)
        gl.glActiveTexture(gl.GL_TEXTURE1)
        gl.glBindTexture(gl.GL_TEXTURE_2D, input_img_id)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_S, gl.GL_CLAMP_TO_EDGE)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_T, gl.GL_CLAMP_TO_EDGE)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_LINEAR)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MIN_FILTER, gl.GL_LINEAR)
        gl.glTexImage2D(gl.GL_TEXTURE_2D, 0, gl.GL_RGBA32F, self.width, self.height, 0, gl.GL_RGBA, gl.GL_FLOAT, None)
        gl.glBindImageTexture(1, input_img_id, 0, gl.GL_FALSE, 0, gl.GL_READ_WRITE, gl.GL_RGBA32F)

        output_img_id = gl.glGenTextures(1)
        gl.glActiveTexture(gl.GL_TEXTURE2)
        gl.glBindTexture(gl.GL_TEXTURE_2D, output_img_id)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_S, gl.GL_CLAMP_TO_EDGE)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_T, gl.GL_CLAMP_TO_EDGE)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_LINEAR)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MIN_FILTER, gl.GL_LINEAR)
        gl.glTexImage2D(gl.GL_TEXTURE_2D, 0, gl.GL_RGBA32F, self.width, self.height, 0, gl.GL_RGBA, gl.GL_FLOAT, None)
        gl.glBindImageTexture(2, output_img_id, 0, gl.GL_FALSE, 0, gl.GL_WRITE_ONLY, gl.GL_RGBA32F)

        seed_img = np.random.random_sample((self.width, self.height, 4))
        seed_img_id = gl.glGenTextures(1)
        gl.glActiveTexture(gl.GL_TEXTURE3)
        gl.glBindTexture(gl.GL_TEXTURE_2D, seed_img_id)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_S, gl.GL_CLAMP_TO_EDGE)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_T, gl.GL_CLAMP_TO_EDGE)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_LINEAR)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MIN_FILTER, gl.GL_LINEAR)
        gl.glTexImage2D(gl.GL_TEXTURE_2D, 0, gl.GL_RGBA32F, self.width, self.height, 0, gl.GL_RGBA, gl.GL_FLOAT, seed_img)
        gl.glBindImageTexture(3, seed_img_id, 0, gl.GL_FALSE, 0, gl.GL_READ_WRITE, gl.GL_RGBA32UI)

        background_img = cv2.imread("museum_of_ethnography_8k.hdr", cv2.IMREAD_UNCHANGED)
        background_img = cv2.cvtColor(background_img, cv2.COLOR_BGRA2RGBA)
        background_img = background_img.reshape(background_img.shape[1], background_img.shape[0], 4)
        background_img_id = gl.glGenTextures(1)
        gl.glActiveTexture(gl.GL_TEXTURE4)
        gl.glBindTexture(gl.GL_TEXTURE_2D, background_img_id)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_S, gl.GL_REPEAT)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_WRAP_T, gl.GL_REPEAT)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_LINEAR)
        gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MIN_FILTER, gl.GL_LINEAR)
        gl.glTexImage2D(gl.GL_TEXTURE_2D, 0, gl.GL_RGBA32F, background_img.shape[0], background_img.shape[1], 0, gl.GL_RGBA, gl.GL_FLOAT, background_img)
        gl.glBindImageTexture(4, seed_img_id, 0, gl.GL_FALSE, 0, gl.GL_READ_ONLY, gl.GL_RGBA32F)

        self.sample = np.array([1], dtype=np.uint)
        self.sample_id = gl.glGenBuffers(1)
        gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, self.sample_id)
        gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, self.sample.nbytes, self.sample, gl.GL_DYNAMIC_DRAW)
        gl.glBindBufferBase(gl.GL_SHADER_STORAGE_BUFFER, 1, self.sample_id);

        scale = np.array([0], dtype=np.float32)
        scale_id = gl.glGenBuffers(1)
        gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, scale_id)
        gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, scale.nbytes, scale, gl.GL_DYNAMIC_DRAW)
        gl.glBindBufferBase(gl.GL_SHADER_STORAGE_BUFFER, 2, scale_id);

        theta = np.array([0], dtype=np.float32)
        theta_id = gl.glGenBuffers(1)
        gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, theta_id)
        gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, theta.nbytes, theta, gl.GL_DYNAMIC_DRAW)
        gl.glBindBufferBase(gl.GL_SHADER_STORAGE_BUFFER, 3, theta_id);

        phi = np.array([0], dtype=np.float32)
        phi_id = gl.glGenBuffers(1)
        gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, phi_id)
        gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, phi.nbytes, phi, gl.GL_DYNAMIC_DRAW)
        gl.glBindBufferBase(gl.GL_SHADER_STORAGE_BUFFER, 4, phi_id);

        moveX = np.array([0], dtype=np.float32)
        moveX_id = gl.glGenBuffers(1)
        gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, moveX_id)
        gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, moveX.nbytes, moveX, gl.GL_DYNAMIC_DRAW)
        gl.glBindBufferBase(gl.GL_SHADER_STORAGE_BUFFER, 5, moveX_id);

        moveY = np.array([0], dtype=np.float32)
        moveY_id = gl.glGenBuffers(1)
        gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, moveY_id)
        gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, moveY.nbytes, moveY, gl.GL_DYNAMIC_DRAW)
        gl.glBindBufferBase(gl.GL_SHADER_STORAGE_BUFFER, 6, moveY_id);


    def render(self):
        while True:
            gl.glClear(gl.GL_COLOR_BUFFER_BIT)

            gl.glBindBuffer(gl.GL_SHADER_STORAGE_BUFFER, self.sample_id)
            gl.glBufferData(gl.GL_SHADER_STORAGE_BUFFER, self.sample.nbytes, self.sample, gl.GL_DYNAMIC_DRAW)
            gl.glUseProgram(self.raytrace_program)
            gl.glDispatchCompute(self.width, self.height, 1)
            gl.glMemoryBarrier(gl.GL_SHADER_IMAGE_ACCESS_BARRIER_BIT)
        
            self.sample[0] += self.sample_per_frame
                
            gl.glUseProgram(self.raster_program)
            gl.glEnableVertexAttribArray(0);
            gl.glVertexAttribPointer(0, 2, gl.GL_FLOAT, False, 0, vertices)
            gl.glDrawArrays(gl.GL_TRIANGLE_STRIP, 0, 4)
                
            framebuffer = gl.glReadPixels(0, 0, self.width, self.height, gl.GL_BGR, gl.GL_UNSIGNED_BYTE)
            framebuffer = cv2.imencode(".jpg", (np.frombuffer(framebuffer, np.uint8)).reshape([self.height, self.width, -1])[::-1], [int(cv2.IMWRITE_JPEG_QUALITY), 85])[1].tostring()
            yield b"--frame\r\nContent-Type: image/jpeg\r\n\r\n" + framebuffer + b"\r\n"

Writing engine.py


In [5]:
%%file vertex_shader.glsl

#version 460 core

in vec2 position_vertices;
out vec2 position_screen;

void main() {
	position_screen = position_vertices * 0.5 + 0.5;
	gl_Position = vec4(position_vertices, 0, 1);
}

Writing vertex_shader.glsl


In [6]:
%%file fragment_shader.glsl

#version 460 core

in vec2 position_screen;
out vec4 color;

layout(binding = 2) uniform sampler2D output_img;

void main() {
	color = texture(output_img, position_screen);
}

Writing fragment_shader.glsl


In [7]:
%%file compute_shader.glsl

#version 460 core

#define POW2(X) ((X) * (X))
#define POW5(X) ((X) * (X) * (X) * (X) * (X))
#define DEPTH_MAX (16)
#define SAMPLE_MAX ($sample_max)
#define DELTA (0.01)
#define PI (3.14159265359)
#define ROOT3 (1.73205080757)
#define ROOT6 (2.44948974278)
#define GROUND (-1.0)

#define BACKGROUND (0)
#define DIFFUSE (1)
#define MIRROR (2)
#define GLASS (3)
#define STONE (4)
#define LIGHT (5)

layout(local_size_x = 1, local_size_y = 1, local_size_z = 1) in;

struct ray {
    vec3 origin;
    vec3 direction;
    vec3 scatter;
    uint depth;
};

struct hit {
    float t;
    vec3 position;
    vec3 normal;
    vec3 scatter;
    vec3 emission;
    uint material;
};

struct sphere {
    vec3 center;
    float radius;
    vec3 scatter;
    vec3 emission;
    uint material;
};

layout(rgba32f, binding = 1) uniform image2D input_img;
layout(rgba32f, binding = 2) uniform image2D output_img;
layout(rgba32ui, binding = 3) uniform uimage2D seed_img;
layout(binding = 4) uniform sampler2D background_img;
layout(std430, binding = 1) buffer sample_buffer {
    uint sampleN;
};
layout(std430, binding = 2) buffer scale_buffer
{
    float scale;
};
layout(std430, binding = 3) buffer theta_buffer
{
    float theta;
};
layout(std430, binding = 4) buffer phi_buffer
{
    float phi;
};
layout(std430, binding = 5) buffer moveX_buffer
{
    float moveX;
};
layout(std430, binding = 6) buffer moveY_buffer
{
    float moveY;
};

ivec3 groupNum = ivec3(
	gl_NumWorkGroups.x * gl_WorkGroupSize.x,
	gl_NumWorkGroups.y * gl_WorkGroupSize.y,
	gl_NumWorkGroups.z * gl_WorkGroupSize.z
);
ivec3 groupIdx = ivec3(
	gl_GlobalInvocationID.x,
	gl_GlobalInvocationID.y,
	gl_GlobalInvocationID.z
);

uvec4 xors;
float rand() {
    uint t = (xors[0] ^ (xors[0] << 11));
    xors[0] = xors[1];
    xors[1] = xors[2];
    xors[2] = xors[3];
    xors[3] = (xors[3] ^ (xors[3] >> 19)) - (t ^ (t >> 18));
    return xors[3] / 4294967295.0f;
}

bool hit_sphere(const in sphere s, const in ray r, inout hit h) {
    vec3 oc = r.origin - s.center;
    float a = dot(r.direction, r.direction);
    float b = dot(oc, r.direction);
    float c = dot(oc, oc) - POW2(s.radius);
    float d = POW2(b) - a * c;
    
    float t;
    if (d > 0)
    {
        t = (-b - sqrt(d)) / a;
        if (0 < t && t < h.t)
        {
            h.t = t;
            h.position = r.origin + t * r.direction;
            h.normal = normalize(h.position - s.center);
            h.scatter = s.scatter;
            h.emission = s.emission;
            h.material = s.material;
            return true;
        }
        t = (-b + sqrt(d)) / a;
        if (0 < t && t < h.t)
        {
            h.t = t;
            h.position = r.origin + t * r.direction;
            h.normal = normalize(h.position - s.center);
            h.scatter = s.scatter;
            h.emission = s.emission;
            h.material = s.material;
            return true;
        }
    }
    
    return false;
}

void background(inout ray r, inout hit h) {
    r.depth = DEPTH_MAX;
    h.emission = texture(background_img, vec2(-atan(r.direction.x, r.direction.z) / (2 * PI), acos(r.direction.y) / PI)).rgb;
}

void diffuse(inout ray r, const in hit h) {
    if (dot(-r.direction, h.normal) < 0) {
        r.depth = DEPTH_MAX;
        r.scatter = vec3(0.0);
        return;
    }

    r.depth++;
    r.direction.y = sqrt(rand());
    float d = sqrt(1 - POW2(r.direction.y));
    float v = rand() * 2 * PI;
    vec3 ex = vec3(1, 0, 0);
    vec3 ey = vec3(0, 1, 0);
    vec3 ez = vec3(0, 0, 1);
    float dx = abs(dot(h.normal, ex));
    float dy = abs(dot(h.normal, ey));
    float dz = abs(dot(h.normal, ez));
    vec3 vy = (dy < dx) ? (dz < dy) ? ez : ey : (dz < dx) ? ez : ex;
    vec3 vx = normalize(cross(vy, h.normal));
    vec3 vz = normalize(cross(vx, h.normal));
    
    r.direction = normalize(vx * d * cos(v) + h.normal * r.direction.y + vz * d * sin(v));
    r.origin = h.position + h.normal * DELTA;
    r.scatter *= h.scatter;
}

void mirror(inout ray r, const in hit h)
{
	if (dot(-r.direction, h.normal) < 0)
	{
        r.depth = DEPTH_MAX;
		return;
	}
    r.depth++;
	r.origin = h.position + h.normal * DELTA;
    r.direction = 2 * dot(-r.direction, h.normal) * h.normal + r.direction;
	r.scatter *= h.scatter;
}

float fresnel(const in float n, const in float u)
{
	const float f0 = POW2((n - 1) / (n + 1));
	return f0 + (1 - f0) * POW5(1 - u);
}

void glass(inout ray r, const in hit h)
{
	float n = 1.5;
	vec3 N;
	float t = dot(-r.direction, h.normal);
	if (t > 0)
	{
		n = 1 / n;
		N = h.normal;
		t = t;
	}
	else
	{
        r.depth++;
		n = n;
		N = -h.normal;
		t = -t;
	}
	if (rand() < fresnel(n, t))
	{
		mirror(r, h);
	}
	else
	{
		r.origin = h.position - N * DELTA; 
        r.direction = n * r.direction + (n * t - sqrt(1 - POW2(n) * (1 - POW2(t)))) * N;
		r.scatter *= h.scatter;
	}
}

void stone(inout ray r, const in hit h)
{
	float n = 1.5;
	vec3 N;
	float t = dot(-r.direction, h.normal);
	if (t > 0)
	{
		n = 1 / n;
		N = h.normal;
		t = t;
	}
	else
	{
		n = n;
		N = -h.normal;
		t = -t;
	}
	if (rand() < fresnel(n, t))
	{
		mirror(r, h);
	}
	else
	{
		diffuse(r, h);
	}
}

void light(inout ray r, const in hit h) {
    r.depth = DEPTH_MAX;
}

vec4 toneMap(const in vec4 color, const in float white) {
    return clamp(color * (1 + color / white) / (1 + color), 0, 1);
}

vec4 gammaCorrect(const in vec4 color, const in float gamma) {
    float c = 1 / gamma;
    return vec4(pow(color.r, c), pow(color.g, c), pow(color.b, c), 0);
}

void main() {
    xors = imageLoad(seed_img, groupIdx.xy);
  
    vec4 color_present = (sampleN == 1) ? vec4(0) : imageLoad(input_img, groupIdx.xy);
  
    const vec3 eye = vec3(0, 0, 18);
  
    const uint n_sphere = 23;
    const sphere ss[n_sphere] = {
        // tower
        {
            vec3(7.5, 6.1 + GROUND, 0),
            0.1,
            vec3(0.75),
            vec3(0),
            STONE,
        },
        {
            vec3(7.5, 5.8 + GROUND, 0),
            0.2,
            vec3(0.75),
            vec3(0),
            STONE,
        },
        {
            vec3(7.5, 5.2 + GROUND, 0),
            0.4,
            vec3(0.75),
            vec3(0),
            STONE,
        },
        {
            vec3(7.5, 4.0 + GROUND, 0),
            0.8,
            vec3(0.75),
            vec3(0),
            STONE,
        },
        {
            vec3(7.5, 1.6 + GROUND, 0),
            1.6,
            vec3(0.75),
            vec3(0),
            STONE,
        },
        // piramid
        {
            vec3(0, 4 * ROOT6 / 3 + 1 + GROUND, 0),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(0, 2.0 * ROOT6 / 3 + 1 + GROUND, 2 * ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(1, 2 * ROOT6 / 3 + 1 + GROUND, -ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(-1, 2 * ROOT6 / 3 + 1 + GROUND, -ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(0, 1 + GROUND, -2 * ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(1, 1 + GROUND, ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(-1.0, 1.0 + GROUND, ROOT3 / 3.0),
            1.0,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(0, 1 + GROUND, 4 * ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(2, 1 + GROUND, -2 * ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        {
            vec3(-2, 1 + GROUND, -2 * ROOT3 / 3),
            1,
            vec3(0.75),
            vec3(0),
            GLASS,
        },
        // rosary
        {
            vec3(-10, 2 + GROUND, 0),
            2,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        {
            vec3(-12, 1 + GROUND, 2),
            1,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        {
            vec3(-13, 0.5 + GROUND, 3),
            0.5,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        {
            vec3(-13.5, 0.25 + GROUND, 3.5),
            0.25,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        {
            vec3(-8.0, 1 + GROUND, -2),
            1,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        {
            vec3(-7, 0.5 + GROUND, -3),
            0.5,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        {
            vec3(-6.5, 0.25 + GROUND, -3.5),
            0.25,
            vec3(0.75),
            vec3(0),
            MIRROR,
        },
        // ground
        {
            vec3(0, -10000 + GROUND, 0),
            10000,
            vec3(0.75),
            vec3(0),
            DIFFUSE,
        }, 
    };

    const mat3 M1 = mat3(
        cos(theta), 0, sin(theta),
        0, 1, 0,
        -sin(theta), 0, cos(theta)
    );

	const mat3 M2 = mat3(
		1, 0, 0,
		0, cos(phi), -sin(phi),
		0, sin(phi), cos(phi)
	);
  
  for (int i = 0; i < SAMPLE_MAX; i++) {
    vec4 color_next = vec4(0);
    
    const vec3 position_screen = {
        float(groupIdx.x + rand()) / groupNum.x * 16 - 8,
        float(groupIdx.y + rand()) / groupNum.y *  9 - 4.5,
        eye.z - 9,
    };

    ray r = {
        M1 * M2 * (eye + vec3(moveX, moveY, scale - 1)),
        M1 * M2 * (normalize(position_screen - eye)),
        vec3(1),
        0,
    };

    hit h = {
        1000,
        vec3(0),
        vec3(0),
        vec3(0),
        vec3(0),
        BACKGROUND,
    };
    
     while (r.depth < DEPTH_MAX) {
        for (int i = 0; i < n_sphere; i++) {
            hit_sphere(ss[i], r, h);
        }

        switch (h.material) {
            case BACKGROUND: background(r, h); break;
            case DIFFUSE: diffuse(r, h); break;
            case MIRROR: mirror(r, h); break;
            case GLASS: glass(r, h); break;
            case STONE: stone(r, h); break;
            case LIGHT: light(r, h); break;
        }

        color_next.rgb += h.emission * r.scatter;

        h.t = 10000;
        h.material = BACKGROUND;
    }
    
    color_present += (color_next - color_present) / (sampleN + i);
  }
  
  imageStore(input_img, groupIdx.xy, color_present);
  imageStore(output_img, groupIdx.xy, gammaCorrect(toneMap(color_present, 1000), 2.2));
  imageStore(seed_img, groupIdx.xy, xors);
}

Writing compute_shader.glsl


In [0]:
from subprocess import Popen
import psutil

try:
    for child in psutil.Process(p.pid).children(recursive=True):
        child.kill()
    p.kill()
except:
    pass

cmd = ["gunicorn", "api:app", "-b", "0.0.0.0:80", "-k", "gevent", "--threads", "5"]
p = Popen(cmd)

In [9]:
from pyngrok import ngrok
import socket

host = socket.gethostbyname(socket.getfqdn(socket.gethostname()))

try:
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels:
        ngrok.disconnect(tunnel.public_url)
except:
    pass

http_url = ngrok.connect(name=host)
https_url = http_url.replace("http", "https")
print(https_url)

https://4016c1b2.ngrok.io


In [10]:
%%file index.html

<img src="$https_url" width="960" height="540">

Writing index.html


In [11]:
from IPython.display import HTML
from string import Template

src = Template(open("index.html").read()).substitute(https_url=https_url)

HTML(src)